In [1]:
from keras.preprocessing import image_dataset_from_directory
from keras.utils import get_file
import os
import pandas as pd
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam, SGD
from keras.applications import ResNet50, VGG19
from keras import layers
from src.nigep import Nigep


In [10]:
dataset_url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
data_dir = get_file(
    origin=dataset_url, fname="dog_cat.zip", cache_dir='.'
)
import zipfile
with zipfile.ZipFile('./datasets/dog_cat.zip', 'r') as zip_ref:
    zip_ref.extractall('./datasets')

folders = ['train', 'validation']
classes = ['cats', 'dogs']
path = r'./datasets/cats_and_dogs_filtered'
mixed_path = r'./dataset/default'

list_of_text = []

for folder in folders:
  for classs in classes:
    for file in os.listdir(f'{path}/{folder}/{classs}'):
        list_of_text.append((f'{mixed_path}/{file}', classs))

df = pd.DataFrame(list_of_text, columns = ['x_data', 'y_data'])
df.to_csv(os.path.join('./datasets', 'dataframe.csv'))

# dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
# data_dir = get_file(
#     origin=dataset_url, fname="flower_photos", untar=True, cache_dir='.'
# )

# folders = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
# path = r'./datasets/flower_photos'
# mixed_path = r'./dataset/default'

# list_of_text = []

# for folder in folders:
#   for file in os.listdir(f'{path}/{folder}'):
#       list_of_text.append((f'{mixed_path}/{file}', folder))

# df = pd.DataFrame(list_of_text, columns = ['x_data', 'y_data'])
# df.to_csv(os.path.join('./datasets', 'dataframe.csv'))

68606236/68606236 [==============================] - 4s 0us/step


In [11]:
! mkdir ./datasets/default
! mv ./datasets/cats_and_dogs_filtered/train/dogs/* ./datasets/default
! mv ./datasets/cats_and_dogs_filtered/validation/dogs/* ./datasets/default
! mv ./datasets/cats_and_dogs_filtered/train/cats/* ./datasets/default
! mv ./datasets/cats_and_dogs_filtered/validation/cats/* ./datasets/default
! rm -rf ./datasets/cats_and_dogs_filtered

# Temporary
! mkdir ./dataset
! mv ./datasets/* ./dataset
! rm -rf ./datasets
! rm -rf ./dataset/dog_cat.zip

# ! mkdir ./datasets/default
# ! mv ./datasets/flower_photos/daisy/* ./datasets/default
# ! mv ./datasets/flower_photos/dandelion/* ./datasets/default
# ! mv ./datasets/flower_photos/roses/* ./datasets/default
# ! mv ./datasets/flower_photos/sunflowers/* ./datasets/default
# ! mv ./datasets/flower_photos/tulips/* ./datasets/default
# ! rm -rf ./datasets/flower_photos

# # Temporary
# ! mkdir ./dataset
# ! mv ./datasets/* ./dataset
# ! rm -rf ./datasets
# ! rm -rf ./dataset/flower_photos.tar.gz

mkdir: ./dataset: File exists


In [2]:
model = Sequential()

# pretrained_model = VGG19(include_top=False,
#                    input_shape=(180,180,3),
#                    pooling='avg',classes=5,
#                    weights='imagenet')

# for layer in pretrained_model.layers:
#         layer.trainable=False

# model.add(pretrained_model)
# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dense(256, activation='relu'))
# model.add(Dense(1))

model.add(Conv2D(32, (5, 5), padding="same", activation="relu", input_shape=(200, 200, 3)),)
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(1, activation="sigmoid"))

alpha = 1e-3

# lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, min_delta=alpha, patience=5, verbose=1)
# erl_stopping = EarlyStopping(patience=3, monitor='val_loss', verbose=1)
# callbacks = [lr_reduce, erl_stopping]

model.build()
model.summary()
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

dataframe = pd.read_csv('./dataset/dataframe.csv')
x_data = dataframe['x_data']
y_data = dataframe['y_data']


nigep = Nigep(
    execution_name='resnet50',
    model=model,
    batch_size=32,
    input_shape=(200, 200),
    x_data=x_data,
    y_data=y_data,
    class_mode='binary',
    k_fold_n=5,
    target_names=['cats', 'dogs'],
    epochs=30,
    # callbacks=callbacks
  )

nigep.execute()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 200, 32)      2432      
                                                                 
 max_pooling2d (MaxPooling2  (None, 100, 100, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 100, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 50, 50, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 50, 50, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 25, 25, 128)       0

KeyboardInterrupt: 